<h1><center>SQL - task</center></h1> 

Ваша компания купила крупный сервис для чтения книг по подписке.

Ваша задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг.

Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Необходимые запросы

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## 1. Выведем первые строки таблиц

#### Таблица books:

In [13]:
query_1 = '''
         SELECT *
         FROM books
         LIMIT 5
        '''
books = pd.io.sql.read_sql(query_1, con = engine)
books

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


#### Таблица authors:

In [3]:
query_2 = '''
         SELECT *
         FROM authors
         LIMIT 5
        '''
authors = pd.io.sql.read_sql(query_2, con = engine)
authors

#### Таблица publishers:

In [4]:
query_3 = '''
         SELECT *
         FROM publishers
         LIMIT 5
        '''
publishers = pd.io.sql.read_sql(query_3, con = engine)
publishers

#### Таблица ratings:

In [5]:
query_4 = '''
         SELECT *
         FROM ratings
         LIMIT 5
        '''
ratings = pd.io.sql.read_sql(query_4, con = engine)
ratings

#### Таблица reviews:

In [6]:
query_5 = '''
         SELECT *
         FROM reviews
         LIMIT 5
        '''
reviews = pd.io.sql.read_sql(query_5, con = engine)
reviews

## 2. Написание запросов

#### 2.1 Посчитаем сколько книг вышло после 1 января 2000 года

In [7]:
query_6 = '''
         SELECT COUNT(book_id) AS number_of_books
         FROM books
         WHERE publication_date > '2000-01-01'
        '''
number_of_books_after_01_01_2000 = pd.io.sql.read_sql(query_6, con = engine)
number_of_books_after_01_01_2000

,number_of_books
0,819


После 1 января 2000 года было опубликовано 819 книг.

#### 2.2 Для каждой книги посчитайте количество обзоров и среднюю оценку

In [8]:
query_7 = '''
         SELECT books.book_id, books.title, COUNT(review_id) AS number_of_reviews, AVG (rating) AS avg_rating
         FROM books
         LEFT JOIN reviews ON reviews.book_id = books.book_id
         LEFT JOIN ratings ON ratings.book_id = books.book_id
         GROUP BY books.book_id
         ORDER BY avg_rating DESC, number_of_reviews DESC
        '''
number_of_reviews_and_avg_rating = pd.io.sql.read_sql(query_7, con = engine)
number_of_reviews_and_avg_rating

,book_id,title,number_of_reviews,avg_rating
0,17,A Dirty Job (Grim Reaper #1),16,5.00
1,553,School's Out—Forever (Maximum Ride #2),12,5.00
2,444,Moneyball: The Art of Winning an Unfair Game,9,5.00
3,347,In the Hand of the Goddess (Song of the Liones...,6,5.00
4,610,Tai-Pan (Asian Saga #2),4,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,12,2.25
996,202,Drowning Ruth,9,2.00
997,371,Junky,4,2.00
998,316,His Excellency: George Washington,4,2.00


Мы посчитали для каждой книги количество обзоров и среднюю оценку. Самый высокий рейтинг (5.0) и самое большое количество пользовательских обзоров (16) - у книги A Dirty Job (Grim Reaper #1)	(book_id = 17). Напротив - самый низкий рейтинг (1.5) и меньше всех пользовательских обзоров (4) у книги Harvesting the Heart	(book_id = 303).


#### 2.3 Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [14]:
query_8 = '''
         SELECT publishers.publisher_id, publishers.publisher, COUNT(books.book_id) as number_of_books
         FROM publishers
         JOIN books ON publishers.publisher_id = books.publisher_id
         WHERE books.num_pages > 50
         GROUP BY publishers.publisher_id
         ORDER BY number_of_books DESC
         LIMIT 1
        '''
pd.io.sql.read_sql(query_8, con = engine)

,publisher_id,publisher,number_of_books
0,212,Penguin Books,42


Издательство Penguin Books (publisher_id = 212) выпустило 42 книги, количество страниц в которых больше 50.

#### 2.4 Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [32]:
query_9 = '''
         SELECT authors.author_id, authors.author, AVG(avg_rating) AS avg_rating_total
         FROM authors
         JOIN books ON books.author_id = authors.author_id
         JOIN(
             SELECT books.book_id, books.title, COUNT(rating_id) AS number_of_ratings, AVG (rating) AS avg_rating
             FROM books
             JOIN ratings ON ratings.book_id = books.book_id
             GROUP BY books.book_id, books.title) AS subq ON subq.book_id = books.book_id
         WHERE number_of_ratings >= 50
         GROUP BY authors.author_id, authors.author
         ORDER BY avg_rating_total DESC
         LIMIT 1
        '''
pd.io.sql.read_sql(query_9, con = engine)

,author_id,author,avg_rating_total
0,236,J.K. Rowling/Mary GrandPré,4.283844


Автор с самой высокой средней оценкой книг (среди авторов, у которых 50 и более оценок) - J.K. Rowling/Mary GrandPré (author_id = 236), ее средняя оценка книг - 4.28.

#### 2.5 Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [11]:
# # Для каждого пользователя найдем количество оценок, которое они поставили
# query_10 = '''
#      SELECT ratings.username, COUNT(ratings.rating_id) as number_of_ratings
#      FROM ratings
#      GROUP BY ratings.username
#      ORDER BY number_of_ratings DESC
#      '''
# pd.io.sql.read_sql(query_10, con = engine)

# # Теперь из всех пользователей выделим только тех, кто поставил больше 50 оценок
# query_11 = '''
#      SELECT *
#      FROM
#      (SELECT ratings.username, COUNT(ratings.rating_id) as number_of_ratings
#      FROM ratings
#      GROUP BY ratings.username
#      ORDER BY number_of_ratings DESC
#      ) AS subq
#      WHERE number_of_ratings > 50
#      '''
# pd.io.sql.read_sql(query_11, con = engine)

# #Теперь найдем для каждого выделенного пользователя найдем кол-во ревью, которые они написали
# query_12 = '''
#      SELECT reviews.username, COUNT(reviews.review_id)
#      FROM reviews
#      JOIN (
#              SELECT *
#      FROM
#      (SELECT ratings.username, COUNT(ratings.rating_id) as number_of_ratings
#      FROM ratings
#      GROUP BY ratings.username
#      ORDER BY number_of_ratings DESC
#      ) AS subq
#      WHERE number_of_ratings > 50
#      ) AS subq_2 ON subq_2.username = reviews.username
#      GROUP BY reviews.username
#      '''
# pd.io.sql.read_sql(query_12, con = engine)

In [12]:
#Найдем среднее число ревью, написанных пользователями, которые оставили больше 50 оценок
query_13 = '''
     SELECT AVG(count)
     FROM (
         SELECT reviews.username, COUNT(reviews.review_id)
         FROM reviews
         JOIN (
                 SELECT *
         FROM
         (SELECT ratings.username, COUNT(ratings.rating_id) as number_of_ratings
         FROM ratings
         GROUP BY ratings.username
         ORDER BY number_of_ratings DESC
         ) AS subq
         WHERE number_of_ratings > 50
         ) AS subq_2 ON subq_2.username = reviews.username
         GROUP BY reviews.username
         ) AS subq_3
     '''
pd.io.sql.read_sql(query_13, con = engine)

,avg
0,24.333333


Среднее количество обзоров пользователей, которые поставили больше 50 оценок равно 24.3.